In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.signal import TransferFunction, bode, lsim, lti
from scipy import signal
from scipy.integrate import solve_ivp
import control as control
from IPython.display import display, Latex, Markdown, Image, Math

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

## Smith-Prädiktor

- lineares zeitverzögertes System im Zeitbereich
  \begin{align*}
      \dot{x}(t) & = \frac{1}{T} x(t) + \frac{K_\text{S}}{T} u(t - T_\text{t})
  \end{align*}
- Darstellung im Bildbereich
  \begin{align*}
      G(s) & = \frac{K_\text{S}}{1 + T s} e^{-s T_\text{t}}
  \end{align*}
- Regelung mittels Smith-Prädiktor zur Kompensation der Totzeit

In [ ]:
imag = Image("../../images/smith.png", width=800)
display(imag)

__Definition Parammeter__

In [ ]:
tSim = np.linspace(0, 500, 501)

In [ ]:
T = 68.21
Tt = 100.0
KS = 0.15

## Simulation offener Kreis

In [ ]:
x0 = [0]
u = lambda t: 1 if t >= 0 else 0

In [ ]:
def process(t, x, u, KS, T):
    dx = -1/ T * x[0] + KS / T * u(t)
    return dx

In [ ]:
def processDelay(t, x, u, KS, T, Tt):
    dx = -1/ T * x[0] + KS / T * u(t - Tt)
    return dx

In [ ]:
res = solve_ivp(process,
                [tSim[0], tSim[-1]],
                x0,
                t_eval=tSim, args=(u, KS, T))

In [ ]:
resDelay = solve_ivp(processDelay,
                     [tSim[0], tSim[-1]],
                     x0,
                     t_eval=tSim, args=(u, KS, T, Tt))

In [ ]:
plt.close()

fig, axes10 = plt.subplots(1, 1, figsize=(10,6))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

axes10.plot(res.t, res.y[0, :], 'C0', label=r"unverzögerte Strecke")
axes10.plot(resDelay.t, resDelay.y[0, :], 'C0--', label=r"verzögert Strecke")

axes11 = axes10.twinx()
axes11.plot(tSim, [u(t) for t in tSim], 'C4')
axes11.plot(tSim, [u(t - Tt) for t in tSim], 'C4--')
axes11.set_ylabel(r"$\tilde{u}_{\mathrm{A}}$ in V", color='C4')
axes11.tick_params(axis='y', colors='C4')

axes10.set_xlabel(r"$t$ in s")
axes10.set_ylabel(r"$y$")

axes10.grid() 

handlesAx, labelsAx = axes10.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.90, 0.775, .15), loc=3,
           ncol=4, mode="expand", borderaxespad=0., framealpha=0.5)
plt.show()

## Simulation geschlossener Kreis

In [ ]:
yRef = 0.15
Kp = 14
Ki = 0.45

In [ ]:
def processCtrl(t, x, yRef, KS, T, Kp, Ki):
    e = yRef - x[0]
    u = Kp * e + Ki * x[1]

    dx = np.zeros(2)
    dx[0] = -1/ T * x[0] + KS / T * u
    dx[1] = Ki * e

    return dx

In [ ]:
def processDelayCtrl(t, x, KS, T, Tt, tSim, uAll):
    u = np.interp(t - Tt, tSim, uAll)

    dx = -1/ T * x[0] + KS / T * u
    
    return dx

In [ ]:
def processDelaySmith(t, x, yRef, KS, T, Tt, Ki, tSim, uAll):
    u = np.interp(t - Tt, tSim, uAll)
    uDF = np.interp(t, tSim, uAll)
    
    dx = np.zeros(2)
    dx[0] = -1/ T * x[0] + KS / T * u
    dx[1] = -1/ T * x[1] + KS / T * uDF
    
    return dx

In [ ]:
output = widgets.Output()

with output:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True, gridspec_kw={'height_ratios': [1, 1]})

plt.subplots_adjust(wspace=0.2, hspace=0.07)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([0, tSim[-1]])
ax2.set_xlim([0, tSim[-1]])
ax1.set_ylim([-0.01, 0.40]) 
ax2.set_ylim([-0.25, 0.25]) 
ax1.grid()
ax2.grid()
ax1.set_ylabel(r"$y$")
ax2.set_ylabel(r"$u$")
ax2.set_xlabel(r"$t$ in s")

yRefSys, = ax1.plot([], [], 'C0', label=r'Referenz')
ySys, = ax1.plot([], [], 'C1', label=r'unverzögert', alpha=0.5)
ySysD, = ax1.plot([], [], 'C2', label=r'verzögert')
uSys, = ax2.plot([], [], 'C1', linestyle='dashed', label=r'unverzögert', alpha=0.5)
uSysD, = ax2.plot([], [], 'C2', linestyle='dashed', label=r'verzögert')

handlesAx, labelsAx = ax1.get_legend_handles_labels()

fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.13, 0.94, 0.7675, .15), loc=3,
           ncol=6, mode="expand", borderaxespad=0., framealpha=0.5)

sliderKp = widgets.FloatSlider(value=14,
                               min=1,
                               max=20,
                               step=1,
                               description=r'$K_{\mathrm{P}}$')
sliderKi = widgets.FloatSlider(value=0.45,
                               min=0.05,
                               max=1,
                               step=0.05,
                               description=r'$K_{\mathrm{I}}$')
checkSmith = widgets.Checkbox(value=False,
                              description='Smith Prädiktor',
                              disabled=False)

def calcSys(yRef, Kp, Ki):
    x0 = [0, 0]
    resCtrl = solve_ivp(processCtrl,
                        [tSim[0], tSim[-1]],
                        x0,
                        t_eval=tSim, args=(yRef, KS, T, Kp, Ki))
    u = Kp * (yRef - resCtrl.y[0, :]) + Ki * resCtrl.y[1, :]
    return resCtrl.t, resCtrl.y[0, :], u

def calcDelay(yRef, Kp, Ki):
    yDelayCtrl = np.zeros(len(tSim))
    uDelayCtrl = np.zeros(len(tSim))
    eSum = 0
    dt = tSim[1] - tSim[0]
    x0 = [0]

    for idx, x in enumerate(tSim[:-1]):
        resDelayCtrl = solve_ivp(processDelayCtrl,
                                 [tSim[idx], tSim[idx+1]],
                                 x0,
                                 args=(KS, T, Tt, tSim[:idx+1], uDelayCtrl[:idx+1]))
        x0 = resDelayCtrl.y.T[-1, :]
        y = resDelayCtrl.y.T[-1, 0]

        e = yRef - y
        eSum += e * dt
        u = Kp * e + Ki * eSum

        yDelayCtrl[idx + 1] = y
        uDelayCtrl[idx + 1] = u
    
    return tSim, yDelayCtrl, uDelayCtrl

def calcSmith(yRef, Kp, Ki):
    yDelaySmith = np.zeros(len(tSim))
    yDelaySmithDF = np.zeros(len(tSim))
    uDelaySmith = np.zeros(len(tSim))

    eSum = 0
    dt = tSim[1] - tSim[0]
    x0 = [0, 0]

    for idx, x in enumerate(tSim[:-1]):
        resDelaySmith = solve_ivp(processDelaySmith,
                                  [tSim[idx], tSim[idx+1]],
                                  x0,
                                  args=(yRef, KS, T, Tt, Ki, tSim[:idx+1], uDelaySmith[:idx+1]))
        x0 = resDelaySmith.y.T[-1, :]
        y = resDelaySmith.y.T[-1, 0]
        yDF = resDelaySmith.y.T[-1, 1]
        yDelaySmith[idx + 1] = y
        yDelaySmithDF[idx + 1] = yDF

        eDF = y - np.interp(tSim[idx+1] - Tt, tSim[:idx+2], yDelaySmithDF[:idx+2])
        e = yRef - (yDF + eDF)
        eSum += e * dt
        u = Kp * e + Ki * eSum

        yDelaySmith[idx + 1] = y
        uDelaySmith[idx + 1] = u

    return tSim, yDelaySmith, uDelaySmith


def calcSystem(_):
    Kp = sliderKp.value
    Ki = sliderKi.value
    yRef = 0.15

    if checkSmith.value:
        tSysD, resYSysD, resUSysD = calcSmith(yRef, Kp, Ki)
    else:
        tSysD, resYSysD, resUSysD = calcDelay(yRef, Kp, Ki)

    tSys, resYSys, resUSys = calcSys(yRef, Kp, Ki)
    
    yRefSys.set_data(tSim, np.ones(len(tSim)) * yRef)
    ySys.set_data(tSys, resYSys)
    ySysD.set_data(tSysD, resYSysD)
    uSys.set_data(tSys, resUSys)
    uSysD.set_data(tSysD, resUSysD)

    ax1.set_ylim([np.min(resYSysD) - 0.05, (np.max(resYSysD) + 0.2)]) 
    ax2.set_ylim([np.min(resUSysD) - 1, (np.max(resUSysD) + 2)]) 

    fig.canvas.draw()

sliderKp.observe(calcSystem, names='value')
sliderKi.observe(calcSystem, names='value')
checkSmith.observe(calcSystem, names='value')

calcSystem(_)

sysControls = HBox([sliderKp, sliderKi, checkSmith])
sysControls.layout = make_box_layout()


controls = HBox([sysControls])
VBox([controls, output], layout=Layout(display='flex', flex_flow='row', justify_content='center', align_items='center'))